In [1]:
from janome.tokenizer import Tokenizer

In [14]:
t = Tokenizer()
s = 'おすすめの料理を教えてください'

In [16]:
for token in t.tokenize(s):
    print(token)

おすすめ	名詞,サ変接続,*,*,*,*,おすすめ,オススメ,オススメ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
料理	名詞,サ変接続,*,*,*,*,料理,リョウリ,リョーリ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
教え	動詞,自立,*,*,一段,連用形,教える,オシエ,オシエ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
ください	動詞,非自立,*,*,五段・ラ行特殊,命令ｉ,くださる,クダサイ,クダサイ


In [1]:
!wget http://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
!unzip 794_ruby_4237.zip
!ls -l sanshiro.txt

--2019-01-26 12:13:25--  http://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
www.aozora.gr.jp (www.aozora.gr.jp) をDNSに問いあわせています... 59.106.13.115
www.aozora.gr.jp (www.aozora.gr.jp)|59.106.13.115|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 301 Moved Permanently
場所: https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip [続く]
--2019-01-26 12:13:25--  https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
www.aozora.gr.jp (www.aozora.gr.jp)|59.106.13.115|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 158711 (155K) [application/zip]
`794_ruby_4237.zip' に保存中

794_ruby_4237.zip   100%[===================>] 154.99K   869KB/s 時間 0.2s       

2019-01-26 12:13:25 (869 KB/s) - `794_ruby_4237.zip' へ保存完了 [158711/158711]

Archive:  794_ruby_4237.zip
  inflating: sanshiro.txt            
-rw-r--r--  1 hiramatsuyuuki  staff  363871  6 19  2014 sanshiro.txt


In [2]:
import codecs
# ファイル読込み、内部表現化
f = codecs.open('sanshiro.txt', "r", "sjis")
text = f.read()
f.close()

# ファイル整形
import re
# ヘッダ部分の除去
text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])



一

　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌を


取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。






In [3]:
from janome.tokenizer import Tokenizer
# Tokenneizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞原型のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
for word in ret:
    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

三四郎
京都
ちょっと
用
ある
降りる
ついで
一
する
目
さめる
女
隣
じいさん
話
始める
いる


In [20]:
#  関数テスト
ret = extract_words('勾配降下法')
for word in ret:
    print(word)

勾配
降下
法


In [4]:
# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100,min_count=5,window=5,iter=100)

In [5]:
# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print(model.__dict__['wv']['世間'])

[-0.563186   -0.11887318  0.66275764 -0.29098827  0.02309809 -0.20922549
  0.23195629  0.5226636  -0.07885285 -0.50527006 -0.66221255 -0.1025684
  0.05737205 -1.0302899  -0.3289083   0.31853     0.12568839 -0.2614803
 -0.39552355 -0.04434584 -0.03857547 -0.88955194  0.15444139 -0.09106922
  0.25068918  0.48266354 -0.8184265  -0.58586913  0.09914121 -0.22208445
 -0.35135096  0.0709054  -0.08293461  0.01704745 -0.4322672   0.46470514
 -0.2754094  -0.31491783  0.24328439 -0.7093739   0.01978837  0.2636638
 -0.16215883  0.6252992  -0.2530013   0.32801667 -0.00191444 -0.13582203
  0.08644591  0.22770953  0.76143605 -0.5578901   0.60276544  0.04574628
 -0.74351424 -0.67008686 -0.8171414   0.00847305  0.21800551  0.19472255
 -0.0561062  -0.79417884  0.80249584  0.18939157 -0.33254972 -0.03130971
 -0.72211105  0.81497306 -0.29064637 -0.2791048  -0.22808504  0.02607691
  0.4599352   0.3358488  -0.41397086  0.18989086  0.5822133   0.26180425
  0.9677959  -0.07952777  0.42871222 -0.6222144  -0.59

In [11]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['私']) 
for item in ret:
    print(item[0], item[1])

遊び 0.4558658003807068
御存じ 0.45559656620025635
あなた 0.44520139694213867
御用 0.4353439509868622
おいで 0.42126962542533875
野々宮 0.419454425573349
思える 0.4158669412136078
めんどう 0.41556546092033386
かた 0.4107964038848877
おっかさん 0.4104591906070709
